## DATA PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
df = pd.read_excel("category1.xlsx",sheet_name= [0,1,2,3,4,5,6,7])

In [3]:
allUserData = df[3]
userData = [df[3], df[4], df[5], df[6], df[7]]

for i in userData[1:]:
    allUserData = allUserData.append(i)

In [4]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','location_name','location_category','my_place','location_id','place_id'], 1)

In [5]:
def tomins(ts):
    a = ts.hour*60 + ts.minute
    return a

allUserData['start_time'] = allUserData['start_time'].apply(tomins)
allUserData['end_time'] = allUserData['end_time'].apply(tomins)
allUserData['duration'] = allUserData['duration'].apply(tomins)

allUserData.tail()

,user_id,weekday,start_time,end_time,duration,category,category_id,location
69,5,5,0,45,45,home,2,1
70,5,5,45,470,425,sleep,3,1
71,5,5,470,490,20,food,1,1
72,5,5,490,565,75,commute,17,99
73,5,5,565,810,245,work,6,3


In [6]:
cols = allUserData.columns
for col in cols:
    print(col)

user_id
weekday
start_time
end_time
duration
category
category_id
location


## Random Forest Model

In [7]:
train_df = allUserData[allUserData['weekday'] < 4]
test_df = allUserData[allUserData['weekday'] >= 4]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 193
Number of testing data is : 246


In [8]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4, 7]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4, 7]].values
y_test = test_df.iloc[:, 6].values

In [9]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [10]:
clf.score(X_test, y_test)

0.8008130081300813

In [11]:
imp = clf.feature_importances_
imp

array([0.07474598, 0.03433832, 0.23517535, 0.21636921, 0.43937114])

In [19]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.59      0.67      0.62        30
           2       0.85      0.99      0.91        72
           3       1.00      0.82      0.90        33
           4       1.00      0.75      0.86         4
           5       0.00      0.00      0.00         6
           6       0.74      1.00      0.85        14
           8       0.00      0.00      0.00         5
           9       0.50      0.11      0.18         9
          10       0.00      0.00      0.00         3
          11       0.33      0.67      0.44         3
          12       0.00      0.00      0.00         0
          13       0.50      0.40      0.44         5
          16       0.50      0.67      0.57         6
          17       0.93      0.96      0.95        55
          18       0.00      0.00      0.00         1

   micro avg       0.80      0.80      0.80       246
   macro avg       0.46      0.47      0.45       246
weighted avg       0.76   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
print(confusion_matrix(y_test,predictions))

[[20  2  0  0  0  0  0  0  0  3  1  0  3  1  0]
 [ 0 71  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  4 27  0  0  1  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  4  0  0  0  0  0  0  0  1  0  1  0  0  0]
 [ 0  0  0  0  0 14  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  1  0  1  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  1  0  1  0  0  1  1  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  2  0  0  0  0  0  2  0  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  0  0  4  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0  0  0 53  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [21]:
predictions

array([ 2,  2,  3,  2, 17,  1, 17,  1, 17,  2, 17,  1, 17,  2,  2,  2,  2,
        3,  2, 17, 13, 17,  2,  2,  3,  1,  2,  2,  2,  2,  3,  2,  2,  2,
        3,  2,  1,  1, 17,  2, 17,  4, 17,  2,  3,  2, 17,  6,  1,  6, 17,
        2, 17,  4, 17,  9,  2,  2,  3,  2, 17,  6,  1,  6, 17, 13, 17,  2,
        2,  3,  2, 17,  6,  1,  6, 17, 13,  1, 17, 17,  2,  2,  3,  2, 17,
        1,  1, 17,  1,  1, 12,  9, 17,  2,  2,  3, 17,  6, 17,  1,  6, 17,
       17,  4, 17,  2,  2,  3, 17,  6,  2,  2,  3,  1,  2,  2,  2, 13, 11,
       17,  2,  2,  3,  2,  6,  2,  2,  3,  2, 17,  6, 17,  2,  2,  3,  2,
        1,  2,  2,  2,  3, 17, 12,  6,  1, 17,  2,  2,  3,  2, 17,  6,  1,
        6, 17,  2, 11,  2,  2,  3,  2, 17,  6, 11, 17,  2,  2,  3, 16, 16,
       17,  1,  1,  1, 17,  1, 11,  2,  2,  3,  2,  2,  2,  2,  6, 11, 17,
        2,  2,  3, 16,  2, 17,  6,  1,  6, 17,  1, 17,  2,  2,  3, 16, 17,
        1, 17, 17, 17,  2,  2,  3, 16, 17,  1,  1, 17,  1, 17,  1,  2,  2,
        3, 16, 17,  1,  1

In [242]:
location = pd.read_csv("location1.csv")
location

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
0,1,121.576000,24.986000,7/22\n\n,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2,c3,c4,c5
1,2,121.443000,24.433000,7/23\n,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2,c3,c4,c5
2,3,121.123000,24.765000,7/24,13:00\n,6,6.487776e+08,NaN,banana,banana1,banana2,banana3,banana4,c1,c2,c3,c4,c5
3,4,121.576829,24.986461,2020/07/23,19:13,5,6.487776e+08,-1.000000,國立政治大學,國立政治大學商學院,萊爾富便利商店,國立政治大學圖書館,國立政治大學商學院圖書分館,university,university,convenience_store,library,library
4,5,121.543825,25.038081,2020/07/23,20:48,5,6.487776e+08,-1.000000,TIMES復興仁愛停車場,Home Hotel Da-An 大安,江南春(江浙菜)Yangtse River-台北福華大飯店,明台產物保險股份有限公司,東方富御精緻料理,parking,library,restaurant,point_of_interest,restaurant
5,6,121.544111,25.039766,20:48,20:48,5,6.487776e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,121.544111,25.039766,2020/07/23,20:48,5,6.487779e+08,-1.000000,Villa.like 悅禾莊園Spa -仁愛二館,無毒農粥寶寶,全家便利商店,Vivafit 歐洲女性快速健身教室,台電大安,spa,grocery_or_supermarket,convenience_store,gym,gym
7,8,121.544352,25.060593,20:48,20:48,5,6.487779e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,121.544352,25.060593,2020/07/23,20:52,5,6.487777e+08,-1.000000,中山國中站,臺灣股票博物館,美國運通信用卡會員服務部,TDCC 臺灣集中保管結算所,台灣美國運通國際股份有限公司,subway_station,museum,bank,finance,point_of_interest
9,10,121.547454,25.064400,20:53,20:53,5,6.487777e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [243]:
stopLoc = location.drop(['location_id','longitude','latitude','category2','category3','category4','category5'], 1)
stopLoc = stopLoc.reset_index(drop=True)
stopLoc.head()

,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1
0,7/22\n\n,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1
1,7/23\n,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1
2,7/24,13:00\n,6,6.487776e+08,NaN,banana,banana1,banana2,banana3,banana4,c1
3,2020/07/23,19:13,5,6.487776e+08,-1.0,國立政治大學,國立政治大學商學院,萊爾富便利商店,國立政治大學圖書館,國立政治大學商學院圖書分館,university
4,2020/07/23,20:48,5,6.487776e+08,-1.0,TIMES復興仁愛停車場,Home Hotel Da-An 大安,江南春(江浙菜)Yangtse River-台北福華大飯店,明台產物保險股份有限公司,東方富御精緻料理,parking


In [245]:
dropList = []

for row in range(1,len(stopLoc)-1):
    res = set(stopLoc.iloc[row,5:10]) & set(stopLoc.iloc[row-1,5:10])
    
    if len(res) > 1:
        dropList.append(row)
    
    if (stopLoc.iloc[row+1,1] == stopLoc.iloc[row,1]): #(stopLoc.iloc[row+1,4] > 3) & 
        dropList.append(row)
    
stopLoc = stopLoc.drop(dropList) 

In [247]:
stopLoc = stopLoc.loc[stopLoc['speed'] < 0]
stopLoc = stopLoc.reset_index(drop = True)
stopLoc

,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1
0,2020/07/23,19:13,5,6.487776e+08,-1.0,國立政治大學,國立政治大學商學院,萊爾富便利商店,國立政治大學圖書館,國立政治大學商學院圖書分館,university
1,2020/07/23,20:52,5,6.487777e+08,-1.0,中山國中站,臺灣股票博物館,美國運通信用卡會員服務部,TDCC 臺灣集中保管結算所,台灣美國運通國際股份有限公司,subway_station
2,2020/07/23,20:54,5,6.487777e+08,-1.0,民權停車場,康乾設計工程,亞商大地股份有限公司,貝克漢團體服,邦迪建材有限公司,parking
3,2020/07/23,21:01,5,6.487784e+08,-1.0,劍南路站,UBIKE服務中心,全家便利商店 美華店,美麗新大直皇家影城,典華幸福機構(大直店),subway_station
4,2020/07/23,21:15,5,6.487779e+08,-1.0,聯合光纖通信United Fiber Optic Communication Inc.,全聯福利中心Pxmart 內湖文湖,大成集團 內湖協調中心,台灣太陽誘電股份有限公司,流行時尚家居(內湖門市店),point_of_interest
5,2020/07/23,21:22,5,6.488640e+08,-1.0,軟訊資訊有限公司,全家便利商店 文湖店,小鳥不吃肉,阿華食堂,7-ELEVEN 金湖門市,electronics_store
6,2020/07/24,13:10,6,6.488640e+08,-1.0,世貿公園,餡老滿,ABC Cooking Studio南港教室,台北南港展覽館1館1F,HP Taiwan 台灣惠普科技股份有限公司,park
7,2020/07/24,23:02,6,0.000000e+00,-1.0,軟訊資訊有限公司,全家便利商店 文湖店,阿華食堂,7-ELEVEN 金湖門市,小鳥不吃肉,electronics_store
